In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Предполагаемые имена моделей и пути к данным
base_model_name = "microsoft/phi-4-mini-instruct"  # Необученная модель
fine_tuned_model_name = "C:/users/emir/models/mymodel"  # Замените на фактический путь к вашей дообученной модели
data_path = "path/to/personachat_holdout.jsonl"  # Замените на путь к вашему файлу PersonaChat

# Загрузка токенизатора (используем токенизатор базовой модели)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Функция для расчета перплексии на PersonaChat
def calculate_perplexity_personachat(model, tokenizer, dataset, device="cuda"):
    """
    Вычисляет перплексию для модели на датасете PersonaChat.

    Args:
        model: Модель для оценки (PyTorch модель).
        tokenizer: Токенизатор для модели (Hugging Face Tokenizer).
        dataset: Датасет PersonaChat (Hugging Face Dataset).
        device: Устройство для вычислений ("cuda" или "cpu").

    Returns:
        float: Перплексия.
    """
    model.to(device)
    model.eval()
    total_loss = 0
    total_tokens = 0

    for dialog in dataset:
        for turn in dialog["turns"]:  # Предполагается, что у вас есть поле "turns"
            text = turn["text"]  # и "text" содержит реплику
            inputs = tokenizer(text, return_tensors="pt").to(device)
            labels = inputs["input_ids"]
            with torch.no_grad():
                outputs = model(**inputs, labels=labels)
            loss = outputs.loss.item()
            total_loss += loss * labels.size(1)
            total_tokens += labels.size(1)

    mean_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(mean_loss)).item()
    return perplexity

if __name__ == "__main__":
    # Загрузка датасета PersonaChat
    holdout_dataset = load_dataset("json", data_files=data_path, split="train") # Или как у вас

    # 1. Расчет перплексии для базовой модели
    print("Загрузка базовой модели...")
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name).to("cuda")
    base_perplexity = calculate_perplexity_personachat(base_model, tokenizer, holdout_dataset)
    print(f"Перплексия базовой модели ({base_model_name}): {base_perplexity}")
    del base_model  # Освобождаем память
    torch.cuda.empty_cache()

    # 2. Расчет перплексии для дообученной модели
    print("Загрузка дообученной модели...")
    try:
        fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name).to("cuda")
    except Exception as e:
        print(f"Ошибка при загрузке дообученной модели.  Убедитесь, что путь '{fine_tuned_model_name}' указан правильно, и модель там есть. Ошибка: {e}")
        exit()
    fine_tuned_perplexity = calculate_perplexity_personachat(fine_tuned_model, tokenizer, holdout_dataset)
    print(f"Перплексия дообученной модели ({fine_tuned_model_name}): {fine_tuned_perplexity}")
    del fine_tuned_model #Освобождаем память
    torch.cuda.empty_cache()

Перплексия дообученной модели (microsoft/phi-4-mini-instruct): 87.42
Перплексия дообученной модели (C:\Users\emir\models\mymodel): 26.87
